In [ ]:
# !pip3 install numpy
# !pip3 install pandas
# !pip3 install csv
# !pip3 install openpyxl

# Main Program

## Load the Data

In [123]:
import pandas as pd
import numpy as np
import os
import re

north_path = r"datasets\พจนานุกรมภาษาถิ่นเหนือ.xlsx"
isan_path = r"datasets\พจนานุกรมภาษาถิ่นอีสาน.xlsx"
south_path = r"datasets\พจนานุกรมภาษาถิ่นใต้.xlsx"
orst_path = r"datasets\ราชบัณฑิตฯ"

def load_sheets(file_path, exclude_sheets):
    all_sheets = pd.ExcelFile(file_path).sheet_names
    importing_sheets = [sheet for sheet in all_sheets if sheet not in exclude_sheets]
    return pd.read_excel(file_path, sheet_name=importing_sheets, header=None)

north = load_sheets(north_path, ['อักษรย่อชนิดคำ', 'Example', 'note'])
# isan = pd.read_excel(isan_path, sheet_name=None, header=None)
# south = pd.read_excel(south_path, sheet_name=None, header=None)

def load_excel(directory):
    excel_files = {}
    for filename in os.listdir(directory):
        name = os.path.splitext(filename)[0]
        excel_files[name] = pd.read_excel(os.path.join(directory, filename))
    return excel_files

# orst = load_excel(orst_path)

## Inspect the Data

In [124]:
def inspect(df, num=3):
  for key, value in df.items():
    print(f"{key}\n{df[key].head(num)}\n")

inspect(north)
# inspect(orst)

# inspect(south)
# inspect(isan)

ข
        0                        1    2
0  ช่อง 1                        2    3
1  ขกอยาก            ก. กระโดดขึ้น  NaN
2      ขง  น. กรุง เมืองอาณาบริเวณ  NaN

ก
        0                        1    2    3
0  ช่อง 1                        2    3  4.0
1      กก  น. โคน อย่าง โคนต้นไม้   NaN  NaN
2     NaN     น. ชื่อเรียกนกเงือก   NaN  NaN

ฃ
         0                                             1
0  ฃาบอยาบ  ก. เซ่นผี สังเวยผี เซ่นผีบรรพชนในพิธีแต่งงาน

ค
         0                                                  1    2
0       ค็                              สัน. ก็ แล้ว จึง ย่อม  NaN
1  ค็ข้าแล     คำลงท้ายแสดงความเคารพเมื่อสามัญชนพูดกับเสนาบดี  NaN
2     ค็ดี  สัน. ก็ดี แสดงความหมายเป็นส่วนๆ หรือเน้น ความใ...  NaN

ฅ
           0           1
0   ฅนน้ำโตะ  น. คนบ้ายอ
1  ฅนน้ำสุ่ย  น. คนบ้ายอ
2      ฅอออม    น. คอพอก

ง
     0                            1       2
0   งก  ว. ทุยโหนกใช้กับของขนาดใหญ่     NaN
1   งง                      ว. โค้ง     NaN
2  NaN                       

## Prepocess

In [125]:
def cleaning(df):
    removed = ['ก็ว่า', 'ก็เรียก']
    for key in df.keys():
        # north

        df[key] = df[key][~df[key].apply(lambda row: any(string in str(val) for string in removed for val in row), axis=1)]
        df[key] = df[key].dropna(how='all')
        df[key] = df[key].drop(df[key].columns[2:], axis=1)

        # orst

        # df[key] = df[key].drop(columns = [col for col in [
        #     "subword", "related_word", "word_related"] if col in df[key].columns])
        # df[key] = df[key][:-1] # remove "."

cleaning(north)
# cleaning(orst)
# cleaning(south)
# cleaning(isan)

inspect(north, 10)
# inspect(orst, 10)
# inspect(south, 10)
# inspect(isan, 10)

ข
                0                                                  1
0          ช่อง 1                                                  2
1          ขกอยาก                                      ก. กระโดดขึ้น
2              ขง                            น. กรุง เมืองอาณาบริเวณ
3             NaN     น. กรง - สิ่งที่ทำเป็นซี่ๆ สำหรับขัง สัตว์เล็ก
4             NaN  ก. กลัด - อาการที่เลือดหรือหนองคั่งอยู่ในผิวหร...
6          ขงขวาย                                   ก. แสวงหา เสาะหา
8   ขงเลือดขงหนอง                                 ว. ช้ำเลือดช้ำหนอง
9          ขงหนอง                                        ก. กลัดหนอง
10           ขจิต                                  ก. ประดับ, ตกแต่ง
12             ขด               ล. เรียกของที่เป็นวง เช่น เชือก ๒ ขด

ก
        0                                                  1
0  ช่อง 1                                                  2
1      กก                            น. โคน อย่าง โคนต้นไม้ 
2     NaN                               น. ชื่อเรียกน

## Split POS

In [126]:
# original
pattern = r"(น\.|ก\.|ว\.|สัน\.|สำนวน\.|ล\.|ส\.|อ\.)"
"""
น. --> นาม
ก. --> กริยา
ว. --> วิเศษณ์
สัน. --> สันธาน
สำนวน. --> สำนวน
ล. --> ลักษณนาม
ส. --> สรรพนาม
อ. --> อุทาน
"""

def split_pos_and_definition(df):
    def split_row(row):
        text = ' '.join(str(val) for val in row if pd.notna(val))

        # replace POS
        text = re.sub(r'ลัก\.', r'ล.', text)
        text = re.sub(r'สรรพ\.', r'ส.', text)
        # text = re.sub(r'คำอุทาน\.', r'อ.', text)
        # text = re.sub(r'อุทาน\.', r'อ.', text)

        # remove alternative forms
        text = re.sub(r'\(ดู\s*-\s*.+\)$', '', text)
        text = re.sub(r'\sดู\s.+', '', text)
        text = re.sub(r'"([^"]*)"', r'\1', text)
        text = re.sub(r'\(ดูเพิ่มเติมที่ .*\)', '', text)

        matches = list(set(re.findall(pattern, text)))

        if len(matches) < 1:
            # print(f"old_text: {text}")
            text = ''
            # print(f"new_text: {text}")
            
        # if len(matches) > 1:
        #     print(matches)

        #     # match_str = r'\b(' + '|'.join(re.escape(p) for p in matches) + r') (?=(' + '|'.join(re.escape(p) for p in matches) + r'))'
        #     new_text = text

        #     print(f"text: {text}")
        #     print(f"new_text: {new_text}")

        #     print(matches[0])
        #     print(matches[1])
        #     match_str = r'\b(' + '|'.join(re.escape(p) for p in matches) + r') (?=(' + '|'.join(re.escape(p) for p in matches) + r'))'
        #     print(match_str)
 
        # typos
        text = text.replace("“", "\"").replace("”", "\"").replace("''", "\"").replace("‘", "'").replace("’", "'")

        parts = re.split(pattern, text)
        word = parts[0].strip()
        pos = parts[1].strip() if len(parts) > 1 else ''

        definition = ' '.join(parts[2:]).strip() if len(parts) > 2 else ''
        definition = definition.strip()
        
        return pd.Series([word, pos, definition])

    for key in df.keys():
        df[key] = df[key].apply(split_row, axis=1)
        df[key].columns = ['Word', 'POS', 'Definition']
    return df

north = split_pos_and_definition(north)
# orst = split_pos_and_definition(orst)

inspect(north, 60)
# inspect(orst, 30)

ข
             Word POS                                         Definition
0                                                                       
1          ขกอยาก  ก.                                         กระโดดขึ้น
2              ขง  น.                               กรุง เมืองอาณาบริเวณ
3                  น.        กรง - สิ่งที่ทำเป็นซี่ๆ สำหรับขัง สัตว์เล็ก
4                  ก.  กลัด - อาการที่เลือดหรือหนองคั่งอยู่ในผิวหรือแ...
6          ขงขวาย  ก.                                      แสวงหา เสาะหา
8   ขงเลือดขงหนอง  ว.                                    ช้ำเลือดช้ำหนอง
9          ขงหนอง  ก.                                           กลัดหนอง
10           ขจิต  ก.                                     ประดับ, ตกแต่ง
12             ขด  ล.                  เรียกของที่เป็นวง เช่น เชือก ๒ ขด
13                 ก.                                         ม้วนเป็นวง
14                 ก.                                   ขยดเขยิบ กระเถิบ
15      ขดขององอด  ก.                            

In [ ]:
# test

def split_pos_and_definition(df):
    pattern = r"(น\.|ก\.|ว\.|สัน\.|สำนวน\.|ล\.|ส\.|อ\.)"
    expanded_rows = []

    def split_row(row):
        text = ' '.join(str(val) for val in row if pd.notna(val))

        text = re.sub(r'ลัก\.', r'ล\.', text)
        text = re.sub(r'สรรพ\.', r'ส\.', text)
        text = re.sub(r'\(ดู\s*-\s*.+\)$', '', text)
        text = re.sub(r'\sดู\s.+', '', text)
        text = re.sub(r'"([^"]*)"', r'\1', text)

        matches = set(re.findall(pattern, text))
        if len(matches) > 1:
            segments = re.findall(rf".*?{pattern}.*?(?={pattern}|$)", text)
            for segment in segments:
                segment = segment.strip()
                parts = re.split(pattern, segment)
                word = parts[0].strip() if parts else ''
                pos = parts[1].strip() if len(parts) > 1 else ''
                definition = ' '.join(parts[2:]).strip() if len(parts) > 2 else ''
                expanded_rows.append([word, pos, definition])
        else:
            parts = re.split(pattern, text)
            word = parts[0].strip()
            pos = parts[1].strip() if len(parts) > 1 else ''
            definition = ' '.join(parts[2:]).strip() if len(parts) > 2 else ''
            expanded_rows.append([word, pos, definition])
        
        return expanded_rows
    
    for key in df.keys():
        df[key] = df[key].apply(split_row, axis=1)
        df[key].columns = ['Word', 'POS', 'Definition']
    return df

north = split_pos_and_definition(north)
inspect(north, 30)

In [127]:
def fill_blank_words(df):
    for key in df.keys():
        df[key].dropna(how='all', inplace=True)
        df[key].replace("", np.nan, inplace=True)
        df[key].ffill(axis=0, inplace=True)
    return df

[fill_blank_words(north) for _ in range(2)]

inspect(north, 40)
# inspect(orst, 40)

ข
             Word POS                                         Definition
1          ขกอยาก  ก.                                         กระโดดขึ้น
2              ขง  น.                               กรุง เมืองอาณาบริเวณ
3              ขง  น.        กรง - สิ่งที่ทำเป็นซี่ๆ สำหรับขัง สัตว์เล็ก
4              ขง  ก.  กลัด - อาการที่เลือดหรือหนองคั่งอยู่ในผิวหรือแ...
6          ขงขวาย  ก.                                      แสวงหา เสาะหา
8   ขงเลือดขงหนอง  ว.                                    ช้ำเลือดช้ำหนอง
9          ขงหนอง  ก.                                           กลัดหนอง
10           ขจิต  ก.                                     ประดับ, ตกแต่ง
12             ขด  ล.                  เรียกของที่เป็นวง เช่น เชือก ๒ ขด
13             ขด  ก.                                         ม้วนเป็นวง
14             ขด  ก.                                   ขยดเขยิบ กระเถิบ
15      ขดขององอด  ก.                                         งอก่องอขิง
16         ขดฅ้าย  ก.                            

## Output

In [128]:
def save_dataframes_to_csv(dataframes, region, output_path):
    for sheet_name, df in dataframes.items():
        file_path = f"{output_path}/new_{region}_{sheet_name}.csv"
        df.to_csv(file_path, index=False, encoding='utf-8-sig')

regions = {
    'north': north,
    # 'isan': isan,
    # 'south': south,
    # 'orst': orst
    }

for region, dataframes in regions.items():
    save_dataframes_to_csv(dataframes, region, r"output\csv_test2")

# Test

In [26]:
text = "นกแอ่นลม	(ดู นกแอ่นฟ้า)"
matches = list(set(re.findall(pattern, text)))
if len(matches) < 1:
    text = np.nan

text

nan

1 load
2 inspect
3 each region
3.1 north
    - copy headword
    - remove alternative row
3.2 isan
3.3 south
3.4 orst

# Outline

### south

***
1st problem -- number \๑ \๒ \๓, remove "." after each row and before next meaning

sample data:
ก็กแก็ก	ว. ๑. เล็ก ๆ น้อย ๆ, เหลาะแหละ (ภ.ก.). ๒. เสียงดังเบา ๆ

desired output:
ก็กแก็ก	ว. เล็ก ๆ น้อย ๆ, เหลาะแหละ (ภ.ก.)
ก็กแก็ก	ว. เสียงดังเบา ๆ

***
2nd problem -- misalignment, remove "." after "ก็เรียก" and "ก็ว่า"

sample data:
กง	น. ๑. กง, ไม้รูปโค้งที่ตั้งเป็นโครงเรือ	
	กงข้าง	น. กงเสริมของเรือ
	กงโค้ง, กงแซม, กงหนาบ	ก็เรียก.
	กงวาน	น. กงเรือที่มีรูให้น้ำถ่ายเทไปได้ตลอดท้องเรือ
		๒. ชื่อสัตว์สี่เท้าสะเทินน้ำสะเทินบก ตระกูลเดียวกับคางคก ตัวโต มีจุดแดงบริเวณหัว อาศัยตามป่าในช่วงฤดูผสมพันธุ์ต้วผู้จะเปลี่ยนสีผิวหนังเป็นสีแดง ผิวหนังมีพิษ
	กงเขา, โก้งโค้ง, ค้างคกไฟ, โคตรค้างคก, โจงโคร่ง (ส.ฎ.), ผลักโก้งโค้ง (น.ศ.)	ก็เรียก.
		๓. ชื่อพืชคลุมดินชนิดหนึ่งคล้ายว่านหางช้าง มี ๒ ชนิด คือ กงแดงกับกงขาว. ๔. ชื่อพืชน้ำ ขึ้นตามทุ่งนา หรือพื้นที่ชุ่มน้ำ. ๕. ผ้าคลุมศีรษะของชาวไทยมุสลิมใช้ในพิธีละหมาด (น.ธ.). ๖. ส่วนรอบของล้อรถหรือเกวียน เรียก "กงล้อ". ๗. ไร่ล้มลุกที่ถางเป็นป่าหย่อม ๆ ตามเนื้อที่และกั้นเป็นขอบเขตไว้ เช่น หักล้างถางกงในดงดิน (สุวรรณวงศา). ๘. ชื่อขนมชนิดหนึ่งใช้ทำบุญในวันสารทไส้ถั่วเขียวบดละเอียดผสมน้ำตาล ชุบแป้ง ปั้นเป็นรูปต่าง ๆ แล้วทอดน้ำมัน

desired output:
กง	น.	ไม้รูปโค้งที่ตั้งเป็นโครงเรือ
กงข้าง	น.	กงเสริมของเรือ, กงโค้ง, กงแซม, กงหนาบ ก็เรียก
กงวาน	น.	กงเรือที่มีรูให้น้ำถ่ายเทไปได้ตลอดท้องเรือ
กงวาน	น.	ชื่อสัตว์สี่เท้าสะเทินน้ำสะเทินบก ตระกูลเดียวกับคางคก ตัวโต มีจุดแดงบริเวณหัวอาศัยตามป่าในช่วงฤดูผสมพันธุ์ต้วผู้จะเปลี่ยนสีผิวหนังเป็นสีแดง ผิวหนังมีพิษ, กงเขา, โก้งโค้ง, ค้างคกไฟ, โคตรค้างคก, โจงโคร่ง (ส.ฎ.), ผลักโก้งโค้ง (น.ศ.) ก็เรียก
กงวาน	น.	ชื่อพืชคลุมดินชนิดหนึ่งคล้ายว่านหางช้าง มี ๒ ชนิด คือ กงแดงกับกงขาว
กงวาน	น.	ชื่อพืชน้ำ ขึ้นตามทุ่งนา หรือพื้นที่ชุ่มน้ำ
กงวาน	น.	ผ้าคลุมศีรษะของชาวไทยมุสลิมใช้ในพิธีละหมาด (น.ธ.)
กงวาน	น.	ส่วนรอบของล้อรถหรือเกวียน เรียก "กงล้อ"
กงวาน	น.	ไร่ล้มลุกที่ถางเป็นป่าหย่อม ๆ ตามเนื้อที่และกั้นเป็นขอบเขตไว้ เช่น หักล้างถางกงในดงดิน (สุวรรณวงศา)
กงวาน	น.	ชื่อขนมชนิดหนึ่งใช้ทำบุญในวันสารทไส้ถั่วเขียวบดละเอียดผสมน้ำตาล ชุบแป้ง ปั้นเป็นรูปต่าง ๆ แล้วทอดน้ำมัน

### isan

while south use "ก็เรียก" more frequently, isan use "ก็ว่า", tho both be solved at the same time

if one of these words exists in the first row, the first row should be removed = [
	"ก็เรียก",
	"ก็ว่า"
	"ช่อง 1",
	"1",
	"2",
	"3",
	"แม่คำ",
	"ลูกคำ/ ความหมาย",
	"ความหมายลูกคำ",
	"คำสื่อ",
	"สอบถามพี่ ๆ อีกที",
	"ลูกคำ",
	"คำคล้ายลูกคำ",
	"ความหมายลูกคำ คำสื่อ",
	"ตรวจสอบกับตัวเล่ม / ถามตอง"
]

notable_outliers that must be removed (I cannot name all of them and you shouldn't hard-cod either), you must think how to remove this
[
	"พจนานุกรมเขียน "รักอย่างลำเอียง.ล" จึงลบ ".ล" ออกแล้ว (น.366)",
	"(น.250) เดิมในพจนานุกรม ระหว่าง เขียนต่อ นิยามของ ระแวดระวัง จึงแยกออกมาเป็นแม่คำ และใส่ชนิดของคำ",
	"หนังสือพิมพ์ตก ความหมาย เป็น ความหาย น. ๒๐๔"
]

sample data:
กง ๑	น. สิ่งที่เป็นวงกลม หรือขอบ เช่น กงเกียน = กงเกวียน		
	กงกวัก	น. เครื่องมือสำหรับกรอด้าย, กรอไหม	
	กงแก้ว	น. ดวงชะตา	
		กงชาตา	ก็เรียก 
	กงไกว	น. สว่าน	
	กงชาตา	ดู กงแก้ว	
กง ๒	น. เขต, แดน, บริเวณ		
	กงราฐ	น. เขตเมือง, บริเวณเมือง	
	กงโลก	น. จักรวาล	
		ขงโลก	ก็เรียก

desired output:
กง	น.	สิ่งที่เป็นวงกลม หรือขอบ เช่น กงเกียน = กงเกวียน
กง	น.	เขต, แดน, บริเวณ
กงกวัก	น.	เครื่องมือสำหรับกรอด้าย, กรอไหม
กงแก้ว	น.	ดวงชะตา, กงชาตา	ก็เรียก
กงไกว	น.	สว่าน
กงราฐ	น. เขตเมือง, บริเวณเมือง
กงโลก	น. จักรวาล, ขงโลก

as you can see the "กงชาตา	ดู กงแก้ว" is removed because it's redundant
also the second "กง" is rearranged, i think you can alphabetically sort (it should be noted that i'm not expert in thai, i might've sorted this "desired output", the ideal is to A-Z, or ก-ฮ)

### north

in addition to isan's removed list [
	"แม่คำ", "ลูกคำ/ ความหมาย",	"ความหมายลูกคำ","คำสื่อ","สอบถามพี่ ๆ อีกที","ลูกคำ","คำคล้ายลูกคำ","ความหมายลูกคำ คำสื่อ",	"ตรวจสอบกับตัวเล่ม / ถามตอง"
]

north's removed list is [
	"ช่อง 1",
	"1",
	"2",
	"3",
]

reminder, if in one of these elements in this removed list exists in X row, X row must be removed

sample data:
ช่อง 1	2	3	4
กก	น. โคน อย่าง โคนต้นไม้ 		
	น. ชื่อเรียกนกเงือก 		
	น. ชื่อแม่น้ำใน จังหวัดเชียงราย		
	น.ชื่อหญ้าชนิดหนึ่งใช้ทำเลื่อ เรียกว่าเลื่อกก		
	ก. สับ บั่น ฟัน ตัด เซ่น กก เกง สำนวน. กกหางปล่อยวัด กกหัวกกหาง		

desired output:
กก	น.	โคน อย่าง โคนต้นไม้
กก	น.	ชื่อเรียกนกเงือก
กก	น.	ชื่อแม่น้ำใน จังหวัดเชียงราย
กก	น.	ชื่อหญ้าชนิดหนึ่งใช้ทำเลื่อ เรียกว่าเลื่อกก
กก	ก.	สับ บั่น ฟัน ตัด เซ่น กก เกง
กก	สำนวน.	กกหางปล่อยวัด กกหัวกกหาง

as you can see, an incorrect row is removed, misaligned data is rearranged, when "ก." and "สำนวน." exists, it should be like this